# Lecture 03: Boundary conditions in OpenFOAM

<hr style="border:2px solid gray"> </hr>

## Authors

Ajay B Harish$^+$, Sanjay Govindjee and Frank McKenna <br>
*SimCenter, University of California, Berkeley* <br>

$^+$**Corresponding author** <br>
*Web:* www.ajaybharish.com <br>
*Email:* ajaybh@berkeley.edu <br>

## How to cite
Harish, Ajay Bangalore; Govindjee, Sanjay; McKenna, Frank (2020) “CFD Notebooks (Beginner).” DesignSafe-CI. https://doi.org/10.17603/ds2-w2x6-nm09.


## Copyright

The CFD Notebooks are developed by the NHERI <a href="http://simcenter.designsafe-ci.org">SimCenter</a>. It is copyrighted to “The Regents of the University of California” and is licensed under the BSD 2-clause license. For more information, please check our webpage for information on <a href="https://nheri-simcenter.github.io/CFD-Notebooks/geninfo/copyright.html">licensing and distribution</a>.

## Goal of this lecture

- What is the boundary conditions in OpenFOAM?
- How do I specify the boundary conditions in OpenFOAM?
- Basic boundary conditions in OpenFOAM
- Can boundary conditions vary with time?
- **Estimated time for completion:** 30-60 minutes

For any queries, write to us on our <a href="http://simcenter-messageboard.designsafe-ci.org/smf/index.php?board=11.0">message board</a>.

<b>If you would like our feedback, you can post your solutions for the exercises to the <a href="http://simcenter-messageboard.designsafe-ci.org/smf/index.php?board=11.0">message board</a></b>.

In [28]:
# Introductory video for lecture 03 (L03a)
from IPython.display import IFrame
display(IFrame(('https://www.youtube.com/embed/5VAoRlRp1eA?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

<hr style="border:2px solid gray"> </hr>

## Step 1/5: Boundary conditions in OpenFOAM

The topic related to the treatment of boundaries in OpenFOAM is quite complex. They related the physics and the geometry of the problem and form an integral part of the solution and numerics. We will re-visit boundary conditions again in our intermediate lectures once we have had a comprehension regarding mesh etc.

In [29]:
# Introduction to boundary conditions in OpenFOAM (L03b)
display(IFrame(('https://www.youtube.com/embed/w1dNHd5dsE0?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

### First thoughts:

While OpenFOAM can be used to solve problems in solids, electromagnetics and even financial modeling, most often it is used as a solver for fluid mechanics problems. In this regard, we can believe that our goal is to solve the Navier-Stokes equations where the unknowns are velocities and pressure.

In order to solve our PDE, we need to specify both initial and boundary conditions. In the previous lecture, we saw that the `system` folder contains dictionaries that specify the interpolation schemes used. In this lecture, we will learn how to specify the velocities, pressures and other quantities (like turbulence parameters) at the boundaries.

At the moment, the question that haunts us are: 

- What are the type of boundary conditions that exist in OpenFOAM?
- Where and how do we specify the boundary conditions? 
- Can I write my own boundary conditions?

We will try to discuss each of these in detail as we go ahead.

<hr style="border:2px solid gray"> </hr>

## Step 2/5: Specifying boundary conditions in OpenFOAM

There are two aspects in specifying the boundary conditions in OpenFOAM.

- First is related to specify the geometric region where the boundary condition need to be specified
- Specifying the physical quantities (velocity, pressure, etc.) on these geometric regions

In this section, let us open the files and check how to modify them to specify the boundary conditions.

In [31]:
# Video to explain specifying boundary conditions (L03c)
display(IFrame(('https://www.youtube.com/embed/iliKBx9-gt8?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

Before moving ahead, let us open the `blockMesh` dictionary file in the `system` folder.

In [25]:
f = open("examples/cavity/system/blockMeshDict", "r")
file_contents = f.read()
print(file_contents)

/*--------------------------------*- C++ -*----------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     | Website:  https://openfoam.org
    \\  /    A nd           | Version:  8
     \\/     M anipulation  |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      blockMeshDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

convertToMeters 0.1;

vertices
(
    (0 0 0)
    (1 0 0)
    (1 1 0)
    (0 1 0)
    (0 0 0.1)
    (1 0 0.1)
    (1 1 0.1)
    (0 1 0.1)
);

blocks
(
    hex (0 1 2 3 4 5 6 7) (20 20 1) simpleGrading (1 1 1)
);

edges
(
);

boundary
(
    movingWall
    {
        type wall;
        faces
        (
            (3 7 6 2)
        );
    }
    fixedWalls
    {
        type wall;
        faces
        (
          

### Define the geometry for boundary

The first aspect is to find the geometric regions where the boundary conditions need to be defined. We will briefly address this here and much more in the next lecture on `blockMesh`. As we scroll through the file, we can see the keyword `boundary`. We will discuss the concept of patches, again in the next lecture on meshing using `blockMesh` utility. However, here it is important to note that inside the boundary, a combination of faces of the mesh are used to specify the `type` for the faces. The `type` are specified as a `patch / wall / empty`. Here,

- `patch` - refers to the geometric region where a boundary condition can be specified
- `wall` - refers to a solid wall and the turbulence models specify wall functions near these faces

### Define the values for the boundary

The more important concern for us in this lecture is related to the definition of the values for the boundaries. Boundary condition values are specified in field files, e.g.  `p`, `U`, in time directories (like `0` folder). The information in the `0` directory is also the initial conditions to be considered when solving the set of equations. Additionally, more time directories can also be specified if required. Each of these field files, present in the time directories (like the `0` directory) provide an opportunity to add information about the boundary conditions related to the particular field of interest. 

To illustrate this, let us go ahead and open the `U` (i.e. velocity field) sub-dictionary of our cavity example and compare this to the `blockMesh` dictionary that we have opened earlier.

In [26]:
f = open("examples/cavity/0/U", "r")
file_contents = f.read()
print(file_contents)

/*--------------------------------*- C++ -*----------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     | Website:  https://openfoam.org
    \\  /    A nd           | Version:  8
     \\/     M anipulation  |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       volVectorField;
    object      U;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

dimensions      [0 1 -1 0 0 0 0];

internalField   uniform (0 0 0);

boundaryField
{
    movingWall
    {
        type            fixedValue;
        value           uniform (1 0 0);
    }

    fixedWalls
    {
        type            noSlip;
    }

    frontAndBack
    {
        type            empty;
    }
}

// ************************************************************************* //



The `blockMesh` dictionary specified three boundaries, under the keyword `boundary`, namely:

- `frontAndBack`
- `movingWall`
- `fixedWalls`

These are the names that we have assigned to the group of faces clustered below this. We can also find that these three names are available in the `U` sub-dictionary that we have opened just above. While the `blockMesh` dictionary defined where these geometric regions are located, the field-value dictionaries specified their values and parameters.

In the next section, we will discuss the options available for these boundary conditions. 

### Exercise 3.1

Explore the `blockMesh` and field dictionaries for our other example (`pitzDaily`) and list the patches and walls that are used?

<hr style="border:2px solid gray"> </hr>

## Step 3/5: Basic boundary conditions

There are primarily two types of boundary conditions available in OpenFOAM. Additionally, OpenFOAM also allows users to write their own boundary condition, if needed. In this section, we will explore the primary two types of boundary conditions.

In [30]:
# Video to explain basic vs. derived bc (L03d)
display(IFrame(('https://www.youtube.com/embed/valQfptMtIw?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

### Basic boundary conditions

As a beginner, it is easy to think of the boundary conditions as some fixed value assigned to a particular physical field. For ex: In the case of our cavity example, the top wall is moving at a constant speed. Here, we can consider the velocity to be of a fixed or particular value. In general, we can say that any quantity $F$ has a specific value assigned on the patch and that does not change over the course of the simulation.

Similarly, we can also fix the gradient (i.e. the value of $\frac{\partial F}{\partial \mathbf{n}}$), where $\mathbf{n}$ is the normal to the patch. Often, a `zeroGradient` boundary condition is commonly used as well. These boundary conditions are basic and most of the other boundary conditions can be derived as a function of these. Formally, the basic boundary conditions keywords include:


- `fixedValue`: This boundary condition implies a fixed value over the patch.


- `fixedGradient`: This boundary condition implies a fixed value for the gradient over the patch. A particular version of the boundary condition is the `zeroGradient`.


- `mixed`: This boundary condition is generally a mix of fixed value and gradient-type boundary. The switch is often made using a reference value.


- `directionMixed`: This boundary conditions allow different boundary conditions in the normal and tangential directions. 


- `calculated`: This boundary condition is when the boundary condition is calculated as a function of other patch fields and values.

### Exercise 3.2

Let us consider the earlier `pitzDaily` example and open the velocity and pressure field dictionary from the `0` directory. Open them and identify the basic boundary conditions that are used in the example.

In [27]:
f = open("examples/pitzDaily/0/U", "r")
file_contents = f.read()
print(file_contents)

/*--------------------------------*- C++ -*----------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     | Website:  https://openfoam.org
    \\  /    A nd           | Version:  7
     \\/     M anipulation  |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       volVectorField;
    object      U;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

dimensions      [0 1 -1 0 0 0 0];

internalField   uniform (0 0 0);

boundaryField
{
    inlet
    {
        type            fixedValue;
        value           uniform (10 0 0);
    }

    outlet
    {
        type            zeroGradient;
    }

    upperWall
    {
        type            noSlip;
    }

    lowerWall
    {
        type            noSlip;
    }

    frontAndBack
    {
        type            empty;


Alongside the basic boundary condition, there are various other boundary conditions that have been specifically designed for the inlet, outlet, wall etc. of various physical fields (like velocity, pressure etc.). As we walk through our intermediate and advanced tutorials, we will introduce you to more of these boundary conditions. Before we complete this chapter, let us consider one small detour to learn about time-varying boundary conditions and apply them to our lid-driven cavity problem.

<hr style="border:2px solid gray"> </hr>

## Step 4/5: Time-varying boundary conditions

In this last section, we will discuss a special type of boundary conditions that allow variation with time. While `fixedValue` or `fixedGradient` boundary conditions are ideal to start the learning process, most of the realistic problems in nature are time-varying. In order to be prepared to solve more realistic problems, we would like to learn how to impose time-varying boundary conditions.

In [32]:
## Time-varying boundary conditions (L03e)
display(IFrame(('https://www.youtube.com/embed/rfTirgtDJ1Q?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

There are sevaral boundary conditions where an input keyword can be used to specify the variation with time. One of the simplest one is the `uniformFixedValue` . This allows a value to be assigned to the patch and the value can be controlled using keywords for variation in time. These keywords allow variation of the quantity with time as `sine / cosine / polynomial / linear ramp` or generically through a CSV table. Some examples include:

`Constant value`

    inlet
    (

        type         uniformFixedValue;
        uniformValue constant 10; // Implies a constant value of 10 units

    )

`Sinosoidal`

    inlet
    (

        type           uniformFixedValue;
        uniformValue
        {
            type       sine; // Type of variation is sinosoidal 
            frequency  10;   // Frequency of the sine wave
            amplitude  0.1;  // Amplitude of the sine wave
            scale      1;    // Scaling factor for the wave
            offset     0;    // Phase Offset
        }
    
    )

Similarly, the type above can also be replaced as a `square` wave.

`Polynomial`

    inlet
    (
        type         uniformFixedValue
        uniformValue polynomial ((a b) (c d) (e f)) //Implies a*t^b + c*t^d + e*t^f (where t is the time)   
    )

As we go through the tutorial, we will explore more advanced boundary conditions and apply them to problems of interest.

### Exercise 2.4

- We had used a constant velocity for the top wall in our Lid-driven cavity problem. Convert the boundary condition to be a sinosiodal wave by assuming that the velocity of the top wall is now $v_0 \sin\omega t$ where $v_0$ is the original constant velocity of 1 m/s and $\omega = 2\pi$.

**Clue**: Consider the following steps:
- Copy the example file to your data depot
- Modify the `blockMesh` dictionary
- Initialize the agave
- Start the agave job as earlier. Here, note that the files dictionaries are now in your user folder and not the published folder. So the `input directory` in the `jobdetails` will change to `"inputDirectory": "agave://designsafe.storage.default/username/folder-path-to-cavity-example"`
- Run with the start and end times being the same. You can change this in the *controlDict* dictionary file in the `system` folder (remember Lecture 01). This will ensure that the mesh is created but the simulation does not start.


<hr style="border:2px solid gray"> </hr>

## Step 5/5: Recap and next step

In this lecture, we explored the application of boundary conditions in OpenFOAM - including, basic, derived and time-varying boundary conditions. In the advanced modules, we will also discuss writing custom boundary conditions. In the next and last lecture in this beginner series of CFD Notebooks, we will explore the concept of meshing using the `blockMesh` utility. 

In [33]:
## Recap and next steps (L03f)
display(IFrame(('https://www.youtube.com/embed/-X9QSLlyRY4?rel=0&amp;controls=1&amp;showinfo=0'), width=560, height=315))

# References

1. CFD-Notebooks documentation [(Link)](https://nheri-simcenter.github.io/CFD-Notebooks)
2. OpenFOAM Wiki [(Link)](https://openfoamwiki.net/index.php/Main_Page)
3. Official OpenFOAM documentation [(Link)](https://openfoam.org/resources)
4. Documentation from CFD Direct [(Link)](https://cfd.direct/openfoam/documentation)